In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import *
import random
from pyspark.sql import Row
# Criando uma sessão Spark
spark = SparkSession.builder \
    .appName("SQL") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.memory", "3g") \
    .config("spark.driver.memory", "3g")\
    .getOrCreate()

# Criando DataFrame de clientes
schema_clientes = StructType([
    StructField('cliente_id', IntegerType(), True),
    StructField('nome', StringType(), True),
    StructField('idade', IntegerType(), True),
    StructField('cidade', StringType(), True),
    StructField('genero', StringType(), True)
])

clientes_data = [
    (i, 'Cliente' + str(i), random.randint(18, 65),
     random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Brasília']),
     random.choice(['M', 'F']))
    for i in range(1, 1000)
]

df_clientes = spark.createDataFrame(clientes_data, schema=schema_clientes)

# Criando DataFrame de produtos
schema_produtos = StructType([
    StructField('produto_id', IntegerType(), True),
    StructField('nome_produto', StringType(), True),
    StructField('categoria', StringType(), True),
    StructField('preco', FloatType(), True)
   
])

produtos_data = [
    Row(
        produto_id=i,
        nome_produto='Produto' + str(i),
        categoria=random.choice(['Eletrônicos', 'Roupas', 'Alimentos', 'Acessórios']),
        preco=random.uniform(10, 1000),  # Convertendo para float
        disponibilidade=random.choice([True, False])
    )
    for i in range(1, 1000)
]

# Criando o DataFrame de produtos
df_produtos = spark.createDataFrame(produtos_data)

# # Criando DataFrame de vendas
schema_vendas = StructType([
    StructField('venda_id', IntegerType(), True),
    StructField('cliente_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('quantidade', IntegerType(), True),
    StructField('data_venda', StringType(), True)
])

vendas_data = [
    (i, random.randint(1, 100), random.randint(1, 50), random.randint(1, 5), '2023-01-01')
    for i in range(1, 1000)
]

df_vendas = spark.createDataFrame(vendas_data, schema=schema_vendas)

# Criando DataFrame de transações
schema_transacoes = StructType([
    StructField('transacao_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('valor', FloatType(), True),
    StructField('tipo', StringType(), True),
    StructField('status', StringType(), True)
])

transacoes_data = [
    (i, random.randint(1, 50), random.uniform(50, 500),
     random.choice(['Débito', 'Crédito']), random.choice(['Aprovado', 'Pendente', 'Rejeitado']))
    for i in range(1, 1000)
]

df_transacoes = spark.createDataFrame(transacoes_data, schema=schema_transacoes)

23/12/24 16:36:53 WARN Utils: Your hostname, DESKTOP-0KANSSR resolves to a loopback address: 127.0.1.1; using 192.168.189.146 instead (on interface eth0)
23/12/24 16:36:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/24 16:36:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/24 16:36:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Criando visões temporárias para os DataFrames
df_clientes.createOrReplaceTempView("clientes_view")
df_vendas.createOrReplaceTempView("vendas_view")
df_produtos.createOrReplaceTempView("produtos_view")
df_transacoes.createOrReplaceTempView("transacoes_view")

In [3]:
query = """
    SELECT c.nome AS cliente,t.status,
       COUNT(v.venda_id) AS total_vendas,
       AVG(p.preco) AS media_preco,
       COUNT(t.transacao_id) AS total_transacoes
FROM clientes_view c
INNER JOIN vendas_view v ON c.cliente_id = v.cliente_id
LEFT JOIN produtos_view p ON v.produto_id = p.produto_id
LEFT JOIN transacoes_view t ON p.produto_id = t.produto_id
WHERE p.disponibilidade = True
  AND (t.status NOT IN ('Aprovado') OR t.status IS NULL)
GROUP BY c.nome,t.status
HAVING AVG(p.preco) > 400
   AND COUNT(t.transacao_id) > 3;

"""
result = spark.sql(query)
result.show()


In [7]:
df_parcial = df_clientes.alias("c").join(df_vendas.alias("v"), on=col("c.cliente_id") == col("v.cliente_id"))\
    .join(df_produtos.alias('p'), col("v.produto_id")==col("p.produto_id"))\
    .join(df_transacoes.alias('t'), col("p.produto_id")==col("t.produto_id"))\
    .filter((col("p.disponibilidade")==True) & (~col("t.status").isin(["Aprovado"])))\
    .select("c.nome", "v.venda_id", "p.preco", "t.transacao_id", "t.status")

In [8]:
df_parcial.groupBy('nome', 'status')\
    .agg(
        count('venda_id').alias("total_vendas"),
        avg("preco").alias("media_preco"),
        count("transacao_id").alias("total_transacoes")
    )\
    .filter("total_transacoes > 3 and media_preco > 400")\
    .show()

23/12/09 12:12:15 WARN TaskSetManager: Stage 20 contains a task of very large size (2220 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:12:15 WARN TaskSetManager: Stage 21 contains a task of very large size (1175 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:12:16 WARN TaskSetManager: Stage 22 contains a task of very large size (2705 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:12:16 WARN TaskSetManager: Stage 23 contains a task of very large size (1744 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+------------+-----------------+----------------+
|     nome|   status|total_vendas|      media_preco|total_transacoes|
+---------+---------+------------+-----------------+----------------+
|Cliente82|Rejeitado|    35163571| 523.180852500736|        35163571|
|Cliente99|Rejeitado|    35059123|524.9147684735557|        35059123|
|Cliente78|Rejeitado|    34624150|522.3526077709688|        34624150|
| Cliente3|Rejeitado|    35511268|523.8185485095164|        35511268|
|Cliente76|Rejeitado|    35052701|524.2689257934233|        35052701|
| Cliente4|Rejeitado|    35382635|513.3462237628655|        35382635|
|Cliente82| Pendente|    35071692| 523.680556959179|        35071692|
|Cliente48|Rejeitado|    35059161| 522.113491204983|        35059161|
|Cliente94|Rejeitado|    35002647|522.2100555180605|        35002647|
|Cliente19| Pendente|    33429130|523.7999881036228|        33429130|
|Cliente61|Rejeitado|    34550154|522.0394626721434|        34550154|
|Cliente89| Pendente

In [5]:
df_clientes.alias("c")\
    .join(broadcast(df_vendas.alias("v")), on=col("c.cliente_id") == col("v.cliente_id"))\
    .join(broadcast(df_produtos.alias('p')), col("v.produto_id") == col("p.produto_id"))\
    .join(broadcast(df_transacoes.alias('t')), col("p.produto_id") == col("t.produto_id"))\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [produto_id#10L], [cast(produto_id#31 as bigint)], Inner, BuildRight, false
   :- BroadcastHashJoin [cast(produto_id#22 as bigint)], [produto_id#10L], Inner, BuildRight, false
   :  :- BroadcastHashJoin [cliente_id#0], [cliente_id#21], Inner, BuildRight, false
   :  :  :- Filter isnotnull(cliente_id#0)
   :  :  :  +- Scan ExistingRDD[cliente_id#0,nome#1,idade#2,cidade#3,genero#4]
   :  :  +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[1, int, false] as bigint)),false), [id=#651]
   :  :     +- Filter (isnotnull(cliente_id#21) AND isnotnull(produto_id#22))
   :  :        +- Scan ExistingRDD[venda_id#20,cliente_id#21,produto_id#22,quantidade#23,data_venda#24]
   :  +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#654]
   :     +- Filter isnotnull(produto_id#10L)
   :        +- Scan ExistingRDD[produto_id#10L,nome_produto#11,categoria#12,preco#13,dispon

In [6]:
df_clientes.alias("c")\
    .join(df_vendas.alias("v"), on=col("c.cliente_id") == col("v.cliente_id"))\
    .join(df_produtos.alias('p'), col("v.produto_id") == col("p.produto_id"))\
    .join(df_transacoes.alias('t'), col("p.produto_id") == col("t.produto_id"))\
    .show()

+----------+---------+-----+--------+------+--------+----------+----------+----------+----------+----------+------------+----------+-----------------+---------------+------------+----------+---------+-------+---------+
|cliente_id|     nome|idade|  cidade|genero|venda_id|cliente_id|produto_id|quantidade|data_venda|produto_id|nome_produto| categoria|            preco|disponibilidade|transacao_id|produto_id|    valor|   tipo|   status|
+----------+---------+-----+--------+------+--------+----------+----------+----------+----------+----------+------------+----------+-----------------+---------------+------------+----------+---------+-------+---------+
|        53|Cliente53|   57|Brasília|     M|       7|        53|        26|         3|2023-01-01|        26|   Produto26|Acessórios|796.4301535017191|          false|         908|        26|405.86133|Crédito| Pendente|
|        53|Cliente53|   57|Brasília|     M|       7|        53|        26|         3|2023-01-01|        26|   Produto26|Ace